In [1]:
import pandas as pd 
import numpy as np 
from pulp import *

In [2]:
# reading the shifts file 
df=pd.read_csv("Daily_shifts.csv",index_col=0)

In [3]:
df = df.fillna(0).replace("Yes",1)

In [4]:
# It gives the labour reuirement for each time slot and cost of each shift 
df

,Shift 1,Shift 2,Shift 3,Shift 4,Shift 5,Labour required
shifts/Timeslots,,,,,,
9:00-09:15,1,0,0,0,0,50.0
9:15-9:30,1,0,1,0,0,48.0
9:30-9:45,1,0,1,1,0,50.0
9:45-10:00,1,0,1,1,1,50.0
10:00-10:15,0,1,1,1,1,55.0
10:15-10:30,0,1,0,1,1,55.0
10:30-10:45,0,1,0,0,1,30.0
10:45-11:00,0,1,0,0,0,30.0
Shift Cost,20000.00,20000.00,18000.00,18000.00,18000.00,0.0


In [5]:
# Converting Shifts into a matrix
Shifts = df.drop(index=['Shift Cost'], columns=["Labour required"]).values


Defining the variables

In [6]:
# Number of Shifts
n = Shifts.shape[0]
# Numbwe of Slots
S = Shifts.shape[0]
#labour required per shit
l=df.iloc[0:-1,-1].values.astype(float).astype(int)
#cost of each shits
c=df.iloc[-1,0:-1].values.astype(float).astype(int)


In [50]:
#setting the decision variables
x= LpVariable.dicts(name="num_labours",  indices=list(range(n)),lowBound=0,upBound=None, cat="Integer")

In [51]:
x

{0: num_labours_0,
 1: num_labours_1,
 2: num_labours_2,
 3: num_labours_3,
 4: num_labours_4}

In [59]:
# Creating a variable for the problem
prob = LpProblem(name="daily_optimizers", sense=LpMinimize)

In [60]:
#Adding the objective function
prob += lpSum([c[i] * x[i] for i in range(n)])

In [57]:
# Putting in the constrains
for s in range(S):
    prob += lpSum([Shifts[s, j] * x[j] for j in range(n)]) >= l[s]


The problem statement

In [58]:
prob

daily_optimizers:
MINIMIZE
20000*num_labours_0 + 20000*num_labours_1 + 18000*num_labours_2 + 18000*num_labours_3 + 18000*num_labours_4 + 0
SUBJECT TO
_C1: num_labours_0 >= 50

_C2: num_labours_0 + num_labours_2 >= 48

_C3: num_labours_0 + num_labours_2 + num_labours_3 >= 50

_C4: num_labours_0 + num_labours_2 + num_labours_3 + num_labours_4 >= 50

_C5: num_labours_1 + num_labours_2 + num_labours_3 + num_labours_4 >= 55

_C6: num_labours_1 + num_labours_3 + num_labours_4 >= 55

_C7: num_labours_1 + num_labours_4 >= 30

_C8: num_labours_1 >= 30

VARIABLES
0 <= num_labours_0 Integer
0 <= num_labours_1 Integer
0 <= num_labours_2 Integer
0 <= num_labours_3 Integer
0 <= num_labours_4 Integer

In [51]:
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [61]:
print("Shifts","Labour Required")
for shift in range(n):
    print(df.columns[shift],end=" "),print(x[shift].value())
   

Shifts Labour Required
Shift 1 50.0
Shift 2 30.0
Shift 3 0.0
Shift 4 25.0
Shift 5 0.0
